### The task it to predict material demand 3 month in advance starting from March 2017. 

In [ ]:
import pandas as pd
import numpy as np


from keras import backend as K
from keras.layers import Dense
from keras.objectives import mean_absolute_error




from dateutil.relativedelta import relativedelta
import tensorflow as tf
#from keras.backend.tensorflow_backend import set_session

import itertools
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import keras

from keras.models import Sequential,Model
from keras.layers import Input, Dense
from keras.layers import LSTM, SimpleRNN, GRU,GlobalMaxPooling1D,Conv1D,MaxPooling1D, Dropout, Bidirectional
from keras import optimizers
from sklearn.metrics import mean_absolute_error

Preparing the data (load, aggregate)

In [15]:
data = pd.read_csv("data/demand_anonymized_20170802.csv", delimiter=";", parse_dates=["Month"])
eval_set = pd.read_csv('data/eval_correct.csv')
series = data.groupby(["SalOrg", "Material", "Month"])["OrderQty"].sum().reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series.head()
eval_comb = eval_set[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values])) # gets unique combinations

comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])

series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2 = series2.fillna(0)


/Users/oleksiyostapenko/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:

# validation folds
#training on the last 4 month, since othervise lags will be very sparce (go beyond the availiable data)
def create_validation(data, date, m):
    return data[(data["Month"] >= pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) &
                                                                    (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

validation_months = ['2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01']

#preparing validation folds
folds = []
for month in validation_months:
    folds.append(create_validation(series2, month, 4))
    

In [ ]:

train_phase = True
num_lagged = 24
num_predict = 3

def lag_feature(df, colname, lag, num_pred):
    cols_lagged = []
    for i in range(lag, -1, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_fut = []
    for i in range(1, num_pred):
        col_fut_name = '{}(t+{})'.format(colname, i)
        df[col_fut_name] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_fut.append(cols_fut)

    return df, cols_lagged, cols_fut

series, lagged, cols_future = lag_feature(series2, 'OrderQty', num_lagged, num_predict)
series.head()


reframed = series[lagged + cols_future]
reframed.dropna(inplace=True)

idx = reframed.index
values = reframed.values

values = values.astype('float32')

In [ ]:

# fit the MinMax sacler to normalize features, to unscale later same dimentionality should be used i.e. 27 columns
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(values)


In [16]:
def create_model(train_X):
    main_input = Input(shape=(train_X.shape[1], train_X.shape[2]), name='main_input')
    #x = Conv1D(16, 3, padding='valid', activation='relu', strides=1)(main_input)
    #x = Dropout(0.5)(x)
    x = Conv1D(32, 3, padding='valid', activation='relu', strides=1)(main_input)
    x = MaxPooling1D(pool_size = 2)(x)
    x = GRU(32,dropout=0.2,recurrent_dropout=0.2)(x)
    #x = keras.layers.concatenate([x, auxiliary_input])
    output = Dense(3, activation='relu')(x)
    model = Model(inputs=[main_input], outputs=output)
    adam = optimizers.Adam(lr=0.0001)
    model.compile(loss='mae', optimizer=adam)
    return model

In [17]:
class Histories(keras.callbacks.Callback):
    def __init__(self, scaler, val_x, val_y):
        keras.callbacks.Callback.__init__(self)
        self.scaler = scaler
        self.val_x = val_x
        self.val_y = val_y
    
    def on_train_begin(self, logs={}):
        return
    
    def on_train_end(self, logs={}):
        return
    
    def on_epoch_begin(self, epoch, logs={}):
        return
    
    def on_epoch_end(self, epoch, logs={}):
        #if epoch % 3 != 0:
        #    return
        y_pred = self.model.predict(self.val_x)
        temp_x = self.val_x.reshape((self.val_x.shape[0], self.val_x.shape[1]))
        concat = np.concatenate((temp_x,y_pred), axis=1)
        val_unscaled = self.scaler.inverse_transform(concat)
        y_pred = val_unscaled[:, -1 * num_predict:]
        concat = np.concatenate((temp_x,self.val_y), axis=1)
        val_unscaled = self.scaler.inverse_transform(concat)
        y_true = val_unscaled[:, -1 * num_predict:]
        print("Epoch ended")
        print("Test mse ", mean_absolute_error(y_true, y_pred))
        return
    
    def on_batch_begin(self, batch, logs={}):
        return
    
    def on_batch_end(self, batch, logs={}):
        return

## Training the model

In [18]:
num_epochs=3 # change to 50

for fold in folds:
    reframed.index = idx
    train = reframed.loc[fold[0]]
    test = reframed.loc[fold[1]]
    scaled_train = scaler.transform(train)
    scaled_val = scaler.transform(test)
    
    train_X, train_y = scaled_train[:, :-1 * num_predict], scaled_train[:, -1 * num_predict:]
    test_X, test_y = scaled_val[:, :-1 * num_predict], scaled_val[:, -1 * num_predict:]
    
    train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
    test_X = test_X.reshape((test_X.shape[0],test_X.shape[1], 1))
    # reshape input to be 3D [samples, timesteps, features]

    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',
                                  min_delta=0,
                                  patience=2,
                                  verbose=0, mode='auto')
    history = Histories(scaler, test_X, test_y)
    model = create_model(train_X)
    # fit network
    
    if train_phase:
        model.fit(train_X, train_y, epochs = nun_epochs, batch_size = 64, validation_data = (test_X, test_y), verbose = 1, shuffle = False, callbacks = [early_stop, history])


((154704, 24, 1), (154704, 3), (116028, 24, 1), (116028, 3))
Train on 154704 samples, validate on 116028 samples
Epoch 1/3
154624/154704 [============================>.] - ETA: 0s - loss: 9.7602e-04Epoch ended
15.023842377
154704/154704 [==============================] - 162s - loss: 9.7596e-04 - val_loss: 0.0011
Epoch 2/3
154688/154704 [============================>.] - ETA: 0s - loss: 9.3640e-04Epoch ended
14.9210069401
154704/154704 [==============================] - 156s - loss: 9.3631e-04 - val_loss: 0.0011
Epoch 3/3
154624/154704 [============================>.] - ETA: 0s - loss: 7.7053e-04Epoch ended
12.5129446105
154704/154704 [==============================] - 145s - loss: 7.7052e-04 - val_loss: 9.2415e-04
((154704, 24, 1), (154704, 3), (116028, 24, 1), (116028, 3))
Train on 154704 samples, validate on 116028 samples
Epoch 1/3
154624/154704 [============================>.] - ETA: 0s - loss: 8.4428e-04Epoch ended
13.2441935096
154704/154704 [==============================] - 14

## Predict the damend

In [ ]:
test_ind = series[series["Month"]==pd.to_datetime('2017-03-01')].index #need to predict 3 month ahead starting Marh 2017
# in order to scale the data the same dimentionality as before is needed, therefore the cols_future are also taken
test_x = series.loc[test_ind][lagged + cols_future]
test_x = test_x.fillna(0)
values = test_x.values
# ensure all data is float
values = values.astype('float32')
# normalize features
values = scaler.transform(values)
values = values.reshape((values.shape[0], values.shape[1], 1))
pred_X = values[:, :-1 * num_predict]
test_prediction = model.predict(pred_X)

temp_x = pred_X.reshape((pred_X.shape[0], pred_X.shape[1]))

#concatenate with features again in order to unscale the predictions
concat = np.concatenate((temp_x,test_prediction), axis=1)
val_unscaled = scaler.inverse_transform(concat)

#get the unscaled predictions
y_pred = val_unscaled[:, -1 * num_predict:]

tst = series.loc[test_ind].copy()
tst['OrderQty(t+1)'] = y_pred.T[0]
tst['OrderQty(t+2)'] = y_pred.T[1]
tst['OrderQty(t+3)'] = y_pred.T[2]

ml = pd.melt(tst, id_vars=['Material', 'SalOrg'], value_vars=['OrderQty(t+1)', 'OrderQty(t+2)','OrderQty(t+3)'])
ml['date'] = ml.variable.replace({'OrderQty(t+1)':'2017-04', 'OrderQty(t+2)':'2017-05', 'OrderQty(t+3)':'2017-06'})
del ml['variable']
result = eval_set.merge(ml, on=['Material', 'SalOrg', 'date'])
result['demand'] = result['value']
result[['ID', 'demand']].to_csv('rnn.csv', index=False)